In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [ ]:
import pandas as pd

# Assuming the CSV file is in the same directory as your Python script
file_path = "credit_score.csv"

# Read the CSV file into a pandas DataFrame
credit_data = pd.read_csv(file_path)

In [ ]:
credit_data.drop(columns=["CUST_ID"], inplace=True)

In [ ]:
# Extract the categorical variable
cat_gambling = credit_data["CAT_GAMBLING"]

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the categorical variable
encoded_cat_gambling = label_encoder.fit_transform(cat_gambling)

# Replace the original categorical variable with the encoded values
credit_data["CAT_GAMBLING"] = encoded_cat_gambling

In [ ]:
print(credit_data.columns)

Index(['INCOME', 'SAVINGS', 'DEBT', 'R_SAVINGS_INCOME', 'R_DEBT_INCOME',
       'R_DEBT_SAVINGS', 'T_CLOTHING_12', 'T_CLOTHING_6', 'R_CLOTHING',
       'R_CLOTHING_INCOME', 'R_CLOTHING_SAVINGS', 'R_CLOTHING_DEBT',
       'T_EDUCATION_12', 'T_EDUCATION_6', 'R_EDUCATION', 'R_EDUCATION_INCOME',
       'R_EDUCATION_SAVINGS', 'R_EDUCATION_DEBT', 'T_ENTERTAINMENT_12',
       'T_ENTERTAINMENT_6', 'R_ENTERTAINMENT', 'R_ENTERTAINMENT_INCOME',
       'R_ENTERTAINMENT_SAVINGS', 'R_ENTERTAINMENT_DEBT', 'T_FINES_12',
       'T_FINES_6', 'R_FINES', 'R_FINES_INCOME', 'R_FINES_SAVINGS',
       'R_FINES_DEBT', 'T_GAMBLING_12', 'T_GAMBLING_6', 'R_GAMBLING',
       'R_GAMBLING_INCOME', 'R_GAMBLING_SAVINGS', 'R_GAMBLING_DEBT',
       'T_GROCERIES_12', 'T_GROCERIES_6', 'R_GROCERIES', 'R_GROCERIES_INCOME',
       'R_GROCERIES_SAVINGS', 'R_GROCERIES_DEBT', 'T_HEALTH_12', 'T_HEALTH_6',
       'R_HEALTH', 'R_HEALTH_INCOME', 'R_HEALTH_SAVINGS', 'R_HEALTH_DEBT',
       'T_HOUSING_12', 'T_HOUSING_6', 'R_HOUSING',

**EDA**

In [ ]:
 # Extract the target variable
y = credit_data["CREDIT_SCORE"]

# Iterate over each column (excluding the target variable)
for column in credit_data.columns:
    if column != "CREDIT_SCORE":  # Exclude the target variable
        # Extract the feature
        X = credit_data[column]

        # Create a scatter plot
        plt.figure(figsize=(8, 6))
        plt.scatter(X, y, alpha=0.5)
        plt.title(f'Scatter Plot of {column} vs CREDIT_SCORE')
        plt.xlabel(column)
        plt.ylabel('CREDIT_SCORE')
        plt.grid(True)
        plt.show()

NameError: name 'plt' is not defined

**ML Stuff **

In [ ]:
# Separate features (X) and target variable (y)
#X = credit_data.drop(columns=["CREDIT_SCORE"])
X= credit_data[["INCOME", "SAVINGS","DEBT", "CAT_CREDIT_CARD", "CAT_MORTGAGE", "CAT_SAVINGS_ACCOUNT", "CAT_DEPENDENTS"]]
y = credit_data["CREDIT_SCORE"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

# Preprocess the data (scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the Support Vector Machine (SVM) model
svm_model = SVR(kernel='linear')  # You can change the kernel if needed (e.g., 'rbf')
svm_model.fit(X_train_scaled, y_train)

# Predict the target variable on the testing set
y_pred = svm_model.predict(X_test_scaled)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 3443.5241858703102


In [ ]:
# Predict the target variable on the testing set
y_pred = svm_model.predict(X_test_scaled)

# Create a DataFrame to compare predicted vs actual values
comparison_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
})

# Print the first few rows of the comparison DataFrame
print(comparison_df.head(10))
print(X_test.head(10))

   Actual   Predicted
0     638  609.399134
1     532  588.515924
2     606  584.056003
3     732  629.725300
4     635  639.883736
5     475  543.630721
6     563  535.856308
7     601  597.493386
8     371  566.305183
9     512  592.085329
     INCOME  SAVINGS     DEBT  CAT_CREDIT_CARD  CAT_MORTGAGE  \
923   39300   247590    55020                0             0   
921  106933   194424   855466                0             0   
516   58260   267025   233040                0             0   
87   557523  1154870  2349563                1             1   
879  185365   320177   117960                0             1   
463  425002   920837  3577097                1             0   
112  158860   303278  1588598                1             0   
297   45687    18275   260416                0             1   
417   39344    34972   957358                0             0   
658   17875    11917   232374                0             0   

     CAT_SAVINGS_ACCOUNT  CAT_DEPENDENTS  
923       

In [ ]:
param_grid = {
    'C': [0.1, 1, 10],               # Regularization parameter
    'gamma': [0.01, 0.1, 1],         # Kernel coefficient
    'kernel': ['linear', 'rbf']      # Kernel type
}

# Initialize SVR
svm_model = SVR()

# Perform GridSearchCV to find the best combination of hyperparameters
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)

# Get the best hyperparameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict the target variable on the testing set using the best model
y_pred = best_model.predict(X_test_scaled)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
print("Best Hyperparameters:", best_params)
print("Mean Squared Error (on testing set):", mse)

Best Hyperparameters: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
Mean Squared Error (on testing set): 2553.7102796005042


In [ ]:
from joblib import dump
from google.colab import files
# Save the model to disk
dump(svm_model, 'svm_model.joblib')
# Download the file
files.download('svm_model.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Serialize and save the scaler to a file
dump(scaler, 'scaler.joblib')
files.download('scaler.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from joblib import load
svm_model = load('svm_model.joblib')

In [ ]:
prediction = svm_model.predict(scaler.transform([[39300, 247590, 55020,1,1,1,0]]))
print(prediction)

[597.28326621]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
